<br></br>

<div align="center">

<h1 align="center">
    Cuaderno para el trabajo de clasificación relacional
    <br></br>
    Evaluar modelos con la librería node2vec
</h1>

<h6 align="center">
    Antonio Macías Ferrera (antmacfer1@alum.us.es)
    <br></br>
    Delfín Santana Rubio (delsanrub@alum.us.es)
    <br></br>
    Universidad de Sevilla
</h6>

<br></br>

## **Control de Versiones**
    
| **Fecha**  | **Versión** | **Descripción**               |
| :--------- | :---------- | :---------------------------- |
| 04/05/2024 | v1r0        | Primera versión del cuaderno. |
| 07/06/2024 | v1r1        | Correcciones de formato.      |


</div>

<br></br>

## **Índice de contenido**

1. [Introducción](#introducción)
2. [Instanciación del grafo](#creacion)
3. [Evaluación de los modelos](#evaluacion)
    - [Árboles de decisión CART](#cart)
    - [Naive Bayes](#naive-bayes)
    - [KNN](#knn)
    - [Random Forest](#random-forest)
    - [Gradient Boosting](#gradient-boosting)

<br></br>

# <a name="introducción"></a> 1. **Introducción**

En este cuaderno se encuentra todo el codigo neceario para evaluar los modelos de aprendizaje automático aciendo uso de la librería node2vec.

A continuación se importan todas las librerías y métodos necesarios para la ejecución del código.

<br></br>

In [1]:
# Pandas
import pandas as pd

# Numpy
import numpy as np

# Codificadores de sci-kit learn
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

# Train-test split, búsqueda en rejilla, validaciones cruzadas y selección de modelos
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import accuracy_score

# Modelos a entrenar (CART, Naive Bayes, Knn, Support Vector Machines)
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import CategoricalNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Discretizador para Naive Bayes
from sklearn.preprocessing import KBinsDiscretizer

# Normalizador para Knn
from sklearn.preprocessing import MinMaxScaler

# Tubería
from sklearn.pipeline import Pipeline

# Comprobar si un conjunto de datos es linealmente separables (SVM)
from mlxtend.plotting import plot_decision_regions

# <a name="creacion"></a> 2. **Instanciación del grafo**

Aquí se cargan los datos del grafo directamente, sin necesidad de crear el csv ya que node2vec se encarga de procesar el grafo. Después, ya con ayuda del csv se determinan cuales serán los atributos de entrenamiento (las métricas seleccionadas) y el atributo objetivo (tipos de grupos de Facebook).
<br></br>

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import json
import csv

# Step 1: Read the edges.csv file
with open('../musae_facebook_edges.csv', 'r') as f:
    edges_data = csv.reader(f)
    next(edges_data,None)
    edges = [tuple(row) for row in edges_data]

# Step 3: Create a NetworkX graph object
G = nx.Graph()

# Step 4: Add nodes and edges to the graph
G.add_edges_from(edges)

tvshow = []
government = []
politician = []
company = []

# Step 5: Assign features to nodes or edges
with open('../musae_facebook_target.csv', 'r') as f:
    edges_data = csv.reader(f)
    next(edges_data,None)
    for row in edges_data:
        target = row[-1]
        node = row[0]
        if target == "government":
            government.append(node)
        elif target == "tvshow":
            tvshow.append(node)
        elif target == "politician":
            politician.append(node)
        elif target == "company":
            company.append(node)
            
        G.nodes[node]["target"] = target


In [54]:
dim = 32
#dim = 64
#dim = 128
#walk_l = 5 
walk_l = 30
#walk_l = 50
#n_walks = 20
n_walks = 200

In [ ]:
from node2vec import Node2Vec
#n2v = Node2Vec(G, dimensions=32, walk_length=30, num_walks=200, workers=8)
n2v = Node2Vec(G, dimensions=dim, walk_length=walk_l, num_walks=n_walks, workers=8)

In [ ]:
model = n2v.fit(window=10, min_count=1, batch_words=4)

In [ ]:
model.wv.save_word2vec_format("test_walks20_n2v")

In [ ]:
test_n2v = open("test_walks20_n2v","r")
content =test_n2v.read()
test_n2v.close()

content = content.split("\n")[1:]
content = [c.split(" ") for c in content]

array_node_values = []
for c in content:
    if c == [""]:
        continue

    node = c[0]

    
    values = c[1:]
    target = G.nodes[node]["target"]
    values.append(target)
    
    array_node_values.append(values)

with open("test_walks20_n2v.csv","w") as df:
    title = ["val_"+str(x) for x in range(1,dim+1)]
    title.append("target")
    title = ",".join(title)
    df.write(title+"\n")
    for line in array_node_values:
        str_to_write = ",".join(map(str,line))
        df.write(str_to_write+"\n")

    df.close()

In [81]:
values = pd.read_csv('dict_walks20_n2v.csv')
values.head()

,val_1,val_2,val_3,val_4,val_5,val_6,val_7,val_8,val_9,val_10,...,val_24,val_25,val_26,val_27,val_28,val_29,val_30,val_31,val_32,target
0,0.326653,0.245119,0.370400,0.137218,0.076108,-0.232579,0.463081,0.251482,-0.174064,0.340504,...,-0.488282,-0.223379,-0.304842,-0.663589,0.299624,0.414191,-0.321035,-0.404326,-0.193136,government
1,0.320148,0.289716,0.366118,0.167424,0.055731,-0.229751,0.465963,0.232105,-0.187669,0.343839,...,-0.507301,-0.269899,-0.274198,-0.626504,0.294263,0.392915,-0.275156,-0.408714,-0.125305,government
2,0.209439,0.080379,0.842399,0.303018,-0.154328,-0.273266,0.337680,0.138122,-0.471835,-0.176956,...,-0.329429,-0.421791,0.019450,-0.535534,1.051913,0.767207,-0.271064,-0.036868,0.151486,government
3,-0.261546,0.291316,-0.141590,0.289704,0.014588,-0.648793,0.268114,0.184701,-0.208827,-0.387103,...,-0.462350,-0.054326,-0.424857,0.119269,0.219759,0.174394,0.821396,-0.618464,0.358457,company
4,0.327999,0.015874,0.914223,0.480495,0.048015,-0.335717,0.374937,0.188914,-0.332372,-0.008033,...,-0.144513,-0.304243,0.049163,-0.632899,0.922258,0.894132,-0.276315,-0.243167,0.161151,government


In [82]:
atributos_continuos = ["val_"+str(x) for x in range(1,dim+1)]
atributos = values.loc[:, atributos_continuos]
atributos.head()

,val_1,val_2,val_3,val_4,val_5,val_6,val_7,val_8,val_9,val_10,...,val_23,val_24,val_25,val_26,val_27,val_28,val_29,val_30,val_31,val_32
0,0.326653,0.245119,0.370400,0.137218,0.076108,-0.232579,0.463081,0.251482,-0.174064,0.340504,...,0.244488,-0.488282,-0.223379,-0.304842,-0.663589,0.299624,0.414191,-0.321035,-0.404326,-0.193136
1,0.320148,0.289716,0.366118,0.167424,0.055731,-0.229751,0.465963,0.232105,-0.187669,0.343839,...,0.280615,-0.507301,-0.269899,-0.274198,-0.626504,0.294263,0.392915,-0.275156,-0.408714,-0.125305
2,0.209439,0.080379,0.842399,0.303018,-0.154328,-0.273266,0.337680,0.138122,-0.471835,-0.176956,...,0.291465,-0.329429,-0.421791,0.019450,-0.535534,1.051913,0.767207,-0.271064,-0.036868,0.151486
3,-0.261546,0.291316,-0.141590,0.289704,0.014588,-0.648793,0.268114,0.184701,-0.208827,-0.387103,...,0.006707,-0.462350,-0.054326,-0.424857,0.119269,0.219759,0.174394,0.821396,-0.618464,0.358457
4,0.327999,0.015874,0.914223,0.480495,0.048015,-0.335717,0.374937,0.188914,-0.332372,-0.008033,...,0.523425,-0.144513,-0.304243,0.049163,-0.632899,0.922258,0.894132,-0.276315,-0.243167,0.161151


In [83]:
objetivo = values['target']
objetivo.head()

0    government
1    government
2    government
3       company
4    government
Name: target, dtype: object

# <a name="evaluacion"></a> 3. **Evaluación de los modelos**

<br></br>

## <a name="cart"></a> **Árboles de decisión CART**

In [84]:
codificador_objetivo = LabelEncoder()
# El método fit_transform ajusta el codificador a los datos y, a continuación,
# codifica estos adecuadamente. En este caso no necesitamos mantener el
# atributo objetivo como una Series de Pandas.
objetivo = codificador_objetivo.fit_transform(objetivo)
print(f'Clases detectadas: {codificador_objetivo.classes_}')
objetivo

Clases detectadas: ['company' 'government' 'politician' 'tvshow']


array([1, 1, 1, ..., 1, 1, 2])

In [85]:
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos, objetivo, 
    test_size=.2, 
    stratify=objetivo)

rejilla = {
    'max_depth': [3, 5, 8, 12, None],
    'min_samples_split': [2, 120,  200],
    'min_samples_leaf': [1, 7, 20, 40],
    'criterion': ['gini']
}

modelo = DecisionTreeClassifier()

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(estimator=modelo, param_grid=rejilla, cv=10, scoring='accuracy', n_jobs=-1)

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)
mejor_modelo_CART = validaciones_cruzadas.best_estimator_


In [86]:
# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_CART.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"Accuracy: {accuracy}")


Accuracy: 0.8206497552291945


In [87]:
# Seleccionar los atributos más importantes (umbral puede ser ajustado)
selector = SelectFromModel(mejor_modelo_CART, prefit=True, threshold='mean')
atributos_seleccionado = selector.transform(atributos_entrenamiento)

# Entrenar y evaluar el modelo con las mejores características
atributos_entrenamiento_selected = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)
atributos_prueba_selected = selector.transform(atributos_prueba)
mejor_modelo_CART.fit(atributos_entrenamiento_selected, objetivo_entrenamiento)
predicciones_selected = mejor_modelo_CART.predict(atributos_prueba_selected)
accuracy_selected = accuracy_score(objetivo_prueba, predicciones_selected)

print(f"Accuracy con mejores características: {accuracy_selected}")

/usr/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Accuracy con mejores características: 0.7785936804628394


## <a name="naive-bayes"></a> **Naive Bayes**

In [88]:
# Discretizar los atributos para poder usar Naive Bayes
discretizador = KBinsDiscretizer(
    n_bins=4,  # Cada atributo se discretiza en 4 intervalos
    encode='ordinal',  # Los intervalos se codifican numéricamente
    strategy='quantile'  # Cada intervalo contiene la misma cantidad de datos
)

atributos_discretizados = atributos.copy()
atributos_discretizados[atributos_continuos] = discretizador.fit_transform(
    atributos_discretizados[atributos_continuos]
)
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos_discretizados, objetivo, 
    test_size=.2, 
    stratify=objetivo)

# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
rejilla = {
    'alpha': [0.1, 0.5, 1.0, 1.5]
}

modelo = CategoricalNB()

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(estimator=modelo, param_grid=rejilla, cv=5, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Mejor modelo obtenido
mejor_modelo_NB = validaciones_cruzadas.best_estimator_
mejor_modelo_NB

# Evaluar el modelo con los mejores hiperparámetros
predicciones = mejor_modelo_NB.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8026257231864709


In [89]:
# Seleccionar las mejores características (por ejemplo, las 2 mejores)
k_best = 2
selector = SelectKBest(score_func=chi2, k=k_best)
atributos_entrenamiento_selected = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)
atributos_prueba_selected = selector.transform(atributos_prueba)

print("Atributos seleccionados:")
print(atributos_entrenamiento.columns[selector.get_support()])

Atributos seleccionados:
Index(['val_27', 'val_30'], dtype='object')


In [90]:
# Entrenar y evaluar el modelo con las mejores características
mejor_modelo_NB.fit(atributos_entrenamiento_selected, objetivo_entrenamiento)
predicciones_selected = mejor_modelo_NB.predict(atributos_prueba_selected)
accuracy_selected = accuracy_score(objetivo_prueba, predicciones_selected)

print(f"Accuracy con mejores características: {accuracy_selected}")

Accuracy con mejores características: 0.5896751223854028


## <a name="knn"></a> **KNN**

In [91]:
# Establecer división entre atributos de prueba y de entrenamiento 
(atributos_entrenamiento, atributos_prueba,
objetivo_entrenamiento, objetivo_prueba) = train_test_split(
    atributos_discretizados, objetivo, 
    test_size=.2, 
    stratify=objetivo)

# Normalizar atributos
normalizador = MinMaxScaler(feature_range=(0,1))

# Establcer rejillas de hiperparámetros y modelo a evaluar (en este caso, Naive Bayes)
tuberia = Pipeline([('normalizador', normalizador),
                   ('knn', KNeighborsClassifier())])

rejilla = {
    'knn__n_neighbors': [1,3,5,7],
    'knn__metric': ['euclidean', 'manhattan']
}

# Ejecutar Búsqueda en Rejilla con validaciones cruzadas
validaciones_cruzadas = GridSearchCV(tuberia, rejilla, cv=5, scoring='accuracy')

validaciones_cruzadas.fit(atributos_entrenamiento, objetivo_entrenamiento)

# Mejor modelo obtenido
mejor_modelo_Knn = validaciones_cruzadas.best_estimator_
mejor_modelo_Knn

predicciones = mejor_modelo_Knn.predict(atributos_prueba)
accuracy = accuracy_score(objetivo_prueba, predicciones)
print(f"Accuracy: {accuracy}")

Accuracy: 0.9305740987983978


In [92]:
# Selección de características utilizando SelectKBest con chi2
selector = SelectKBest(score_func=chi2, k='all')
selector.fit(atributos_entrenamiento, objetivo_entrenamiento)

k_best = 5
selector = SelectKBest(score_func=chi2, k=k_best)
atributos_entrenamiento_selected = selector.fit_transform(atributos_entrenamiento, objetivo_entrenamiento)
atributos_prueba_selected = selector.transform(atributos_prueba)

mejor_modelo_Knn.fit(atributos_entrenamiento_selected, objetivo_entrenamiento)
predicciones_selected = mejor_modelo_Knn.predict(atributos_prueba_selected)
accuracy_selected = accuracy_score(objetivo_prueba, predicciones_selected)

print(f"Accuracy con mejores características: {accuracy_selected}")

Accuracy con mejores características: 0.6662216288384513
